In [1]:
import pandas as pd
import plotly.graph_objects as go
import networkx as nx

In [3]:
df = pd.read_csv('project_dataset.csv')

df.dropna(subset=['date_start', 'date_end', 'medium_display', 'artwork_type_title', 'department_title', 'artist_title'],
          inplace=True)

df['colorfulness'].fillna(df['colorfulness'].median(), inplace=True)

# We use value_counts() to count the occurrences of each unique value in the "medium_display" column.
class_counts = df["medium_display"].value_counts()

# We select the top 50 biggest classes.
top_classes = class_counts.head(50).index

# We filter the DataFrame to only include rows with medium_display in the top 20 classes.
df_network = df[df["medium_display"].isin(top_classes)]

# We create the network and create the edges
edges = []
for index, row in df_network.iterrows():
    edges.append((row['medium_display'], row['artist_title']))

G = nx.Graph()
G.add_edges_from(edges)
pos = nx.spring_layout(G, seed=3)

# We create the trace of artists and methods and the color
node_trace_artist = go.Scatter(x=[], y=[], text=[], mode='markers', hoverinfo='text', name='artists', marker=dict(color='#a60944', size=[]))
node_trace_method = go.Scatter(x=[], y=[], text=[], mode='markers', hoverinfo='text', name='methods', marker=dict(color='#5d4fa2', size=[]))
edge_trace = go.Scatter(x=[], y=[], line=dict(width=0.5, color='gray'), mode='lines')

for node in G.nodes():
    x, y = pos[node]
    if node in df_network['artist_title'].values:
        node_trace_artist['x'] += (x,)
        node_trace_artist['y'] += (y,)
        node_trace_artist['text'] += (node,)
        node_trace_artist['marker']['size'] += (6,)
    else:
        node_trace_method['x'] += (x,)
        node_trace_method['y'] += (y,)
        node_trace_method['text'] += (node,)
        node_trace_method['marker']['size'] += (10,)

for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_trace['x'] += (x0, x1, None)
    edge_trace['y'] += (y0, y1, None)

# We fix the layout of the network
fig = go.Figure(data=[edge_trace, node_trace_artist, node_trace_method], layout=go.Layout(title={
        'text': 'Artists and Methods',
        'font': {'size': 20, 'family': 'Andale Mono', 'color': 'black'},
        'x': 0.5,
        'y': 0.95,
        'xanchor': 'center',
        'yanchor': 'top'}, showlegend=True, legend=dict(x=1, y=1), hovermode='closest', margin=dict(b=20,l=5,r=5,t=40), xaxis=dict(showgrid=False, zeroline=False, showticklabels=False), yaxis=dict(showgrid=False, zeroline=False, showticklabels=False), plot_bgcolor="white", paper_bgcolor="white"))

fig.update_layout(
    legend=dict(
        font=dict(
            family='Andale Mono',
            size=15,
            color='black'
        )
    )
)
fig.show()